In [ ]:
%pip install numpy opencv-python pandas matplotlib

In [198]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import json
import pandas as pd

Experimenting with Canny Edge Detection Parameters

In [ ]:
def edge_detection(image_path, sigma, low_threshold, high_threshold):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (0, 0), sigmaX=sigma, sigmaY=sigma)
    edges = cv2.Canny(blurred, low_threshold, high_threshold)
    return edges

def process_and_plot_images(input_dir, output_dir, sigma_values, threshold_pairs):
    os.makedirs(output_dir, exist_ok=True)
    image_files = [f for f in os.listdir(input_dir) if f.endswith(('.png', '.jpeg'))]
    
    for image_file in image_files:
        image_path = os.path.join(input_dir, image_file)
        original_image = cv2.imread(image_path)
        original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  
        
        num_cols = len(sigma_values) * len(threshold_pairs) + 1
        plt.figure(figsize=(5 * num_cols, 5))
        plt.subplot(1, num_cols, 1)
        plt.imshow(original_image_rgb)
        plt.title(f"Original Image")
        plt.axis('off')
        plot_idx = 2
        
        for sigma in sigma_values:
            for (low_thresh, high_thresh) in threshold_pairs:
                edges = edge_detection(image_path, sigma, low_thresh, high_thresh)
                
                output_filename = f"{os.path.splitext(image_file)[0]}_edges.png"
                output_path = os.path.join(output_dir, output_filename)
                
                cv2.imwrite(output_path, edges)
                
                plt.subplot(1, num_cols, plot_idx)
                plt.imshow(edges, cmap='gray')
                plt.title(f"σ={sigma}, low={low_thresh}, high={high_thresh}")
                plt.axis('off')
                
                plot_idx += 1
        
        plt.tight_layout()
        plt.show()

# Parameters to experiment with
sigma_values = [0.5, 1, 1.5]
threshold_pairs = [(50, 150), (25, 75), (50, 100), (75, 150)]

input_directory = '/Users/yaseminakin/Desktop/CS484 - Introduction to Computer Vision/HW2/part1/img'   
output_directory = '/Users/yaseminakin/Desktop/CS484 - Introduction to Computer Vision/HW2/part1/edge_img' 

process_and_plot_images(input_directory, output_directory, sigma_values, threshold_pairs)

In [ ]:
book_names = ['automata.jpeg', 'c.jpeg', 'cleancode.jpeg', 'cs.jpeg', 'cv.jpeg',
              'fn.jpeg', 'machineage.jpeg', 'noname.png', 'os.jpeg', 'petrol.jpeg',
              'proofs.jpeg', 'soul.jpeg', 'strip.png', 'topology.jpeg', 'understory.jpeg']

rbook_names = ['rotated_automata.jpeg', 'rotated_c.jpeg', 'rotated_cleancode.jpeg', 'rotated_cs.jpeg', 'rotated_cv.jpeg',
               'rotated_fn.jpeg', 'rotated_machineage.jpeg', 'rotated_noname.png', 'rotated_os.jpeg', 'rotated_petrol.jpeg',
               'rotated_proofs.jpeg', 'rotated_soul.jpeg', 'rotated_strip.png', 'rotated_topology.jpeg', 'rotated_understory.jpeg']

template_path = '/Users/yaseminakin/Desktop/CS484 - Introduction to Computer Vision/HW2/part1/img'
rotate_path = '/Users/yaseminakin/Desktop/CS484 - Introduction to Computer Vision/HW2/part1/rotated_img_new'

os.makedirs(template_path, exist_ok=True)
os.makedirs(rotate_path, exist_ok=True)

def process_images(parameter_set):
    rho = parameter_set['rho']
    theta = parameter_set['theta']
    min_line_length = parameter_set['min_line_length']
    max_line_gap = parameter_set['max_line_gap']
    num_bins = parameter_set['num_bins']
    desired_line_count = parameter_set['desired_line_count']
    max_iterations = parameter_set['max_iterations']

    line_output_dir_original = 'line_images_original'
    line_output_dir_rotated = 'line_images_rotated'
    hist_output_dir_original = 'histograms_original'
    hist_output_dir_rotated = 'histograms_rotated'
    os.makedirs(line_output_dir_original, exist_ok=True)
    os.makedirs(line_output_dir_rotated, exist_ok=True)
    os.makedirs(hist_output_dir_original, exist_ok=True)
    os.makedirs(hist_output_dir_rotated, exist_ok=True)

    template_histograms = {}
    for idx, img_name in enumerate(book_names):
        img_path = os.path.join(template_path, img_name)
        histogram_data = process_single_image(
            image_name=img_name,
            image_path=img_path,
            rho=rho,
            theta=theta,
            min_line_length=min_line_length,
            max_line_gap=max_line_gap,
            num_bins=num_bins,
            desired_line_count=desired_line_count,
            max_iterations=max_iterations,
            line_output_dir=line_output_dir_original,
            hist_output_dir=hist_output_dir_original
        )
        if histogram_data:
            template_histograms[img_name] = histogram_data['histogram']

    rotated_histograms = {}
    for idx, img_name in enumerate(rbook_names):
        img_path = os.path.join(rotate_path, img_name)
        histogram_data = process_single_image(
            image_name=img_name,
            image_path=img_path,
            rho=rho,
            theta=theta,
            min_line_length=min_line_length,
            max_line_gap=max_line_gap,
            num_bins=num_bins,
            desired_line_count=desired_line_count,
            max_iterations=max_iterations,
            line_output_dir=line_output_dir_rotated,
            hist_output_dir=hist_output_dir_rotated
        )
        if histogram_data:
            rotated_histograms[img_name] = histogram_data['histogram']

    matches = match_histograms(template_histograms, rotated_histograms, num_bins)
    return matches

def process_single_image(image_name, image_path, rho, theta, min_line_length, max_line_gap,
                         num_bins, desired_line_count, max_iterations, line_output_dir, hist_output_dir):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not load image: {image_path}")
        return None

    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    blurred_image = cv2.GaussianBlur(gray_image, (0, 0), 0.5)

    edges = cv2.Canny(blurred_image, 50, 100)

    iteration = 0
    adaptive_threshold = desired_line_count
    while iteration < max_iterations:
        lines = cv2.HoughLinesP(edges, rho, theta, threshold=adaptive_threshold,
                                minLineLength=min_line_length, maxLineGap=max_line_gap)
        detected_line_count = len(lines) if lines is not None else 0

        if desired_line_count * 0.8 <= detected_line_count <= desired_line_count * 1.2:
            break
        elif detected_line_count < desired_line_count * 0.8:
            adaptive_threshold = max(int(adaptive_threshold * 0.8), 1)
        else:
            adaptive_threshold = int(adaptive_threshold * 1.2)
        iteration += 1

    line_image_path = os.path.join(line_output_dir, f"{image_name}_lines.png")
    if lines is not None:
        line_image, line_data = draw_lines(lines, gray_image, line_image_path)
    else:
        line_image = cv2.cvtColor(gray_image, cv2.COLOR_GRAY2BGR)
        cv2.imwrite(line_image_path, line_image)
        line_data = []

    histogram, bins = compute_orientation_histogram(line_data, num_bins)
    hist_image_path = os.path.join(hist_output_dir, f"{image_name}_hist_bins{num_bins}.png")
    visualize_histogram(histogram, bins, f"Histogram: {image_name} (Bins={num_bins})", hist_image_path)

    histogram_data = {
        "image_name": image_name,
        "histogram": histogram.tolist(),
        "bins": bins.tolist(),
        "line_data": line_data
    }

    return histogram_data

def draw_lines(lines, original_grayscale_image, line_image_path):
    line_data = []
    line_image = cv2.cvtColor(original_grayscale_image, cv2.COLOR_GRAY2BGR)

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            line_data.append([int(x1), int(y1), int(x2), int(y2)])

    cv2.imwrite(line_image_path, line_image)
    return line_image, line_data

def compute_orientation_histogram(lines, num_bins):
    bins = np.linspace(-np.pi, np.pi, num_bins + 1)
    histogram = np.zeros(num_bins, dtype=np.float32)

    if lines:
        for line in lines:
            x1, y1, x2, y2 = line
            angle = np.arctan2(y2 - y1, x2 - x1)  
            length = np.hypot(x2 - x1, y2 - y1)

            bin_index = np.searchsorted(bins, angle, side='right') - 1
            if bin_index == num_bins:
                bin_index -= 1
            histogram[bin_index] += length

    return histogram, bins

def match_histograms(template_histograms, rotated_histograms, num_bins):
    matches = []
    
    for rotated_name, rotated_hist in rotated_histograms.items():
        min_distance = float('inf')
        best_match = None
        best_shift = 0

        for template_name, template_hist in template_histograms.items():
            for shift in range(num_bins):
                shifted_hist = np.roll(rotated_hist, shift)
                
                distance = np.linalg.norm(template_hist - shifted_hist)
                
                if distance < min_distance:
                    min_distance = distance
                    best_match = template_name
                    best_shift = shift

        rotation_angle = best_shift * (2 * np.pi / num_bins)
        if rotation_angle > np.pi:
            rotation_angle -= 2 * np.pi

        rotated_base_name = rotated_name.replace('rotated_', '')
        is_correct_match = os.path.splitext(rotated_base_name)[0] == os.path.splitext(best_match)[0]

        matches.append({
            'Rotated Image': rotated_name,
            'Matched Template': best_match,
            'Rotation Angle (degrees)': np.degrees(rotation_angle),  
            'Distance': min_distance,
            'Correct Match': is_correct_match
        })

    return matches

def visualize_histogram(histogram, bins, title, output_path):
    plt.figure(figsize=(8, 4))
    bin_centers = (bins[:-1] + bins[1:]) / 2
    plt.bar(bin_centers, histogram, width=np.diff(bins), align='center', edgecolor='black')
    plt.title(title)
    plt.xlabel('Angle (radians)')
    plt.ylabel('Weighted Line Length')
    plt.savefig(output_path)
    plt.close()

def main():
    parameter_set = {
        'rho': 1,
        'theta': np.pi / 180,
        'min_line_length': 10,
        'max_line_gap': 4,
        'num_bins': 90,
        'desired_line_count': 6,
        'max_iterations': 10
    }

    print(f"\nProcessing with parameters: {parameter_set}")
    matches = process_images(parameter_set)

    correct_matches = sum(match['Correct Match'] for match in matches)
    total_matches = len(matches)

    all_results = []
    for match in matches:
        match_result = {
            'Parameters': parameter_set,
            'Rotated Image': match['Rotated Image'],
            'Matched Template': match['Matched Template'],
            'Rotation Angle (degrees)': match['Rotation Angle (degrees)'],
            'Distance': match['Distance'],
            'Correct Match': match['Correct Match']
        }
        all_results.append(match_result)

    print(f"Correct Matches: {correct_matches}/{total_matches}")

    df = pd.DataFrame(all_results)
    df.to_csv('matching_results_single_parameter_set.csv', index=False)
    print("\nAll matching results saved to 'matching_results_single_parameter_set.csv'")

    summary = df.groupby(df['Parameters'].apply(lambda x: tuple(sorted(x.items())))).agg(
        {'Correct Match': 'sum', 'Rotated Image': 'count'}
    ).rename(columns={'Rotated Image': 'Total Matches'})
    print("\nSummary of Results:")
    print(summary)

if __name__ == "__main__":
    main()


Processing with parameters: {'rho': 1, 'theta': 0.017453292519943295, 'min_line_length': 10, 'max_line_gap': 4, 'num_bins': 90, 'desired_line_count': 6, 'max_iterations': 10}
Correct Matches: 7/15

All matching results saved to 'matching_results_single_parameter_set.csv'

Summary of Results:
                                                    Correct Match  \
Parameters                                                          
((desired_line_count, 6), (max_iterations, 10),...              7   

                                                    Total Matches  
Parameters                                                         
((desired_line_count, 6), (max_iterations, 10),...             15  


Function to prepare plots from images in a directory

In [ ]:
def plot_images_from_directory(directory_path, rows=5, cols=3):
    image_files = [f for f in os.listdir(directory_path) 
                   if os.path.isfile(os.path.join(directory_path, f)) 
                   and f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
    
    if not image_files:
        print("No images found in the directory.")
        return
    
    max_images = rows * cols
    image_files = image_files[:max_images]
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 10))
    axes = axes.flatten()  
    
    for idx, ax in enumerate(axes):
        if idx < len(image_files):
            img_path = os.path.join(directory_path, image_files[idx])
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
            
            ax.imshow(img)
            ax.set_title(image_files[idx], fontsize=8)
            ax.axis('off')
        else:
            ax.axis('off')
    
    plt.tight_layout()
    plt.show()

directory_path = "/Users/yaseminakin/Desktop/CS484 - Introduction to Computer Vision/HW2/histograms_rotated" # replace
plot_images_from_directory(directory_path, rows=5, cols=3)